In [66]:
import json
import os

def process_coco(json_path, image_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)
    all_images = os.listdir(image_dir)
    id_image_dict = {
        int(os.path.splitext(image_name)[0]): image_name for image_name in all_images
    }
    res = []
    for item in data['annotations']:
        image_id = item['image_id']
        image_name = id_image_dict[image_id]
        res.append({
            'file_name': image_name,
            'Prompt': item['caption'],
        })
    return res

In [67]:
caption_dir = 'data/coco/annotations/captions_train2017.json'
image_dir = 'data/coco/images/train2017'
output_dir = 'data/coco_train'

In [68]:
r = process_coco(caption_dir, image_dir)

In [69]:
import pandas as pd
df = pd.DataFrame(r)

In [70]:
df.file_name.describe()

count               591753
unique              118287
top       000000052109.jpg
freq                     7
Name: file_name, dtype: object

In [71]:
import shutil
from tqdm import tqdm
os.makedirs('data/coco_train/train', exist_ok=True)
# move image_dir/{file_name} to output_dir/train/{file_name}
for file_name in tqdm(df.file_name.unique()):
    shutil.copy(os.path.join(image_dir, file_name), os.path.join(output_dir, 'train', file_name))

  0%|          | 0/118287 [00:00<?, ?it/s]

 14%|█▎        | 16061/118287 [02:04<13:15, 128.58it/s] 


KeyboardInterrupt: 

In [74]:
with open('data/coco_train/metadata.jsonl', 'w') as f:
    for i, row in df.iterrows():
        f.write(json.dumps(row.to_dict()) + '\n')

In [73]:
df['file_name'] = df['file_name'].apply(lambda x: os.path.join('train', x))